<a href="https://colab.research.google.com/github/bmill42/streaming-data/blob/main/Simple_text_markov_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from Douglas Duhaime's [tutorial](https://www.douglasduhaime.com/posts/making-chiptunes-with-markov-models.html) on Markov models for music.

Import libraries we'll need later and connect Google Drive

In [ ]:
from collections import defaultdict
import random
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Preparing the data

We start by opening the text file containing lines from Shakespeare.

In [ ]:
text = open('/content/drive/MyDrive/COMPFOR 304/Data/tiny-shakespeare.txt').read() # replace path with your own

We want our model to generate entire lines all at once - things like this:

```
Character name: Here are some lines! Now the line will end and a new character will speak.
```

To do this, we'll insert some special words that will serve as **start and end tokens**, which can be used to trigger the beginning or end of the model's output.

Each line spoken by a character in this file is separated by an empty line - the equivalent of typing the 'Return' key twice to start a new paragraph. New lines (technically 'newlines') are represented in plain text with a special character, `\n`, so we can insert or replace newlines by adding or removing `\n` in our text.

We'll take advantage of this by noting that each line is separated from the others by `\n\n`, so anytime we see that character combination we can `replace()` it with the words `END` and `START`. Now each line is demarcated with these special tokens.

The final step is to add the very first `START` and very last `END` at the beginning and end of the file, since our replacement method didn't cover these special cases.

In [ ]:
formatted = text.replace('\n\n', ' END \n\n START ')

training_data = 'START ' + formatted + ' END'

Since the text is all in one long string by default, we need to split it into a list of individual words using the string method `split()`. In machine learning more generally, this process is called **tokenization** - taking a full corpus and turning into the smallest units that you want your model to "understand".

In [ ]:
words = training_data.split(' ')

Then, we create an empty dictionary that will serve as our Markov model by tracking which words follow other words.

We'll use a special kind of dictionary called a `defaultdict`, which works exactly like a normal dictionary with one extra feature: if we try to refer to a key that isn't already in the dictionary, a normal `dict` would give an error, but a `defaultdict` will automatically create that key and give it a default value of whatever type we've specified - in this case, a list.

In [ ]:
next_words = defaultdict(list)

# Building the model

The model itself will consist of a dictionary with one key for each word in the text. The value for each key will be a list of all the words that follow that word in the corpus, including duplicates. By choosing one of those words randomly, the generated two-word sequences will match the frequencies of the sequences in the corpus.

To build the model, we just have to look at every word in the corpus and keep track of which word follows it. In pseudo-code,

```
for each word n,
    add word n+1 to word n's list
```

What this means is that we can't just loop over *every* word in the corpus, because the very last word doesn't have a word `n+1`.

To loop over every word except the last one, we can take advantage of the fact that Python lets us index lists from the *end* using negative indices:

In [ ]:
my_list = [1, 2, 3, 4, 5]
my_list[-1]

This also works for slicing:

In [ ]:
my_list[1:-1] # take the second item through the second-to-last

To build the model, we can just loop over a range covering the length of the list minus one, grabbing the word at the current index along with the next word. Then we add the next word to the list associated with the current word's key in the dictionary.

Recall that the `defaultdict` has the convenient feature of accepting words that it's never seen before as keys by automatically creating a new entry for them.

In [ ]:
for word_index in range(len(words[:-1])):
    current_word = words[word_index]
    next_word = words[word_index+1]

    next_words[current_word].append(next_word)

## Examining the model

Let's peek into the `next_words` dictionary by prompting it with a word:

In [ ]:
next_words['king']

Whenever our Markov model is in the state `'king'`, it will randomly choose the next word using this list.

We can get a better sense of the probabilities involved by sorting the list:

In [ ]:
sorted(next_words['king'])

Clearly `'I'` is much less likely than `'and'`, for example, which seems to have a similar likelihood to `'is`'. Note also that words that include punctuation like commas or periods are separate entries from the non-punctuated versions. We could tweak the preparation of the data to turn these into separate tokens, but the benefit of this version is that punctuation won't appear totally randomly - it will always be attached to words that tend to end sentences or clauses in real text.

Let's examine this slightly more formally and calculate the actual probability for each word given a particular current state.

**Fill out this function** that takes in a list of words (as in the above output) so that it returns a dictionary called `word_freqs` that includes each word and the proportion it represents in the list. E.g.,

```
{
    'I': 0.01,
    'the': 0.10,
    'and': 0.06
}
```

In [ ]:
def word_probabilities(current_word):
    word_freqs = dict()

    # your code here

    return word_freqs

In [ ]:
word_probabilities('king') # test your function here

Once we've created a dictionary associating each word with its probabilities, we can turn it into a dataframe and sort to get the most likely words. If we were to diagram this Markov model, these numbers would be the transition probabilities associated with the arrows pointing to each word:

In [ ]:
pd.DataFrame(word_probabilities('king').items(), columns=['next_word', 'prob']).sort_values('prob', ascending=False).reset_index(drop=True).head(10)

# Exercise: Generating new outputs

Finally, we can sample from the model to generate new Shakespeare-ish lines.

This function will start by randomly choosing a word from the `'START'` state. Since every line in the corpus starts with the name of a character, we should generally get outputs that begin with a character's name, just like in a real play.

From there, the model will continually add the current word to the output string and then choose a new word using the most recent word as the model's state. The process only ends when we see the `'END'` token - which means that our lines should all come to an end eventually, but the exact length depends on the model's random choices.

After testing the function as-is, **modify the `generate_lines` function so that lines can begin with any word provided by the user,** rather than just the `'START'` token. The user-specified start token should be the second argument to the function.

In [ ]:
def generate_line(model):
    output = ''
    word = random.choice(model['START'])

    while word != 'END':
        output += word + ' '
        word = random.choice(model[word])

    print(output.strip() + '\n')
    return

In [ ]:
for i in range(2):
    generate_line(next_words)